In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.insert(0,'src/')
import acs5
import nyserda_acs
import prices
import project_sunroof
import nyserda
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer


%matplotlib inline

In [2]:
acs = nyserda_acs.merging_data()
acs=acs[acs['households']>0]

In [3]:
acs.GEOID10

0       36001000100
1       36001000200
2       36001000300
3       36001000401
4       36001000403
5       36001000404
6       36001000501
7       36001000502
8       36001000600
9       36001000700
10      36001000800
11      36001001100
12      36001001400
13      36001001500
14      36001001600
15      36001001700
16      36001001801
17      36001001802
18      36001001901
19      36001001902
20      36001002000
21      36001002100
22      36001002200
23      36001002300
24      36001002500
25      36001002600
26      36001012700
27      36001012800
28      36001012900
29      36001013000
           ...     
4885    36119014805
4886    36119014806
4887    36119014808
4888    36119014809
4889    36119014810
4890    36119014811
4891    36119014901
4892    36119014903
4893    36119014907
4894    36119014908
4895    36119014909
4896    36119015000
4897    36119981000
4900    36119984000
4902    36121970100
4903    36121970200
4904    36121970300
4905    36121970400
4906    36121970500


In [4]:
np.mean(acs.Project_yes)

0.24715379838542745

In [5]:
a = acs5.Get_census_data()
b = a.get_acs_all()

In [6]:
acs_variables = a.education_percent + a.other_households + a.units_percent + a.heating_percent + a.rooms_percent

In [7]:
acs_variables

['Percent_Less_9th',
 'Percent_12th_no_dip',
 'Percent_HighSchool',
 'Percent_Some_college',
 'Percent_Associate',
 'Percent_Bachelors',
 'Percent_Graduate',
 'Percent_more_than_hs',
 'Avg_hh_size',
 'Median_age',
 'Percent_hh_w_children',
 'Percent_1_unit_detached',
 'Percent_1_unit_attached',
 'Percent_2_units',
 'Percent_3_4_units',
 'Percent_5_9_units',
 'Percent_10_19_units',
 'Percent_20+_units',
 'Percent_10+_units',
 'Percent_1_unit',
 'Percent_Heating_gas',
 'Percent_Heating_gas_tank',
 'Percent_Heating_electricity',
 'Percent_Heating_fuelkero',
 'Percent_Heating_coal_coke',
 'Percent_Heating_wood',
 'Percent_Heating_solar',
 'Percent_Heating_other',
 'Percent_Heating_no',
 'Percent_1_room',
 'Percent_2_rooms',
 'Percent_3_rooms',
 'Percent_4_rooms',
 'Percent_5_rooms',
 'Percent_6_rooms',
 'Percent_7_rooms',
 'Percent_8_rooms',
 'Percent_9_more_rooms',
 'median_rooms']

In [8]:
#Creating new dataframe with only positive cases
temp = acs.loc[acs['Project_yes']==1,:]
#Creating weights for negative class in positive cases
temp.loc[:,'Project_yes'] =0
temp.loc[:,'weight'] = temp.loc[:,'households']-temp.loc[:,'Project_Number']
temp=temp[temp['weight']>0]
#Creating weights for positive class
temp2 = acs.loc[acs['Project_yes']==1,:]
temp2.loc[:,'weight']=temp2.loc[:,'Project_Number']
#Creating weights for negative classe
temp3 =acs.loc[acs['Project_yes']==0,:]
temp3.loc[:,'weight']=temp3.loc[:,'households']
#Concatenating
data = temp.append(temp2.append(temp3, ignore_index=True), ignore_index=True)

/Users/mariamendieta/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/mariamendieta/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


In [9]:
data.describe()

,12th_no_dip,Some_college,Total_households_by_type,Median_income,Bachelors,75k_100k,150k_200k,Pop_over_25,Avg_hh_size,Hh_w_children,...,yearly_sunlight_kwh_w,yearly_sunlight_kwh_f,yearly_sunlight_kwh_median,yearly_sunlight_kwh_total,carbon_offset_metric_tons,existing_installs_count,Mean_price,Var_price,Project_yes,weight
count,6022.000000,6022.000000,6022.000000,5976.000000,6022.000000,6022.000000,6022.000000,6022.000000,6016.000000,6022.000000,...,4.334000e+03,4.334000e+03,4.334000e+03,4.334000e+03,4334.000000,4334.000000,6022.000000,6022.000000,6022.000000,6022.000000
mean,210.570741,465.383926,1536.995018,65091.963186,544.032713,185.634009,99.758718,2837.645965,2.691910,423.684158,...,1.258109e+06,5.394020e+06,1.011119e+04,9.921398e+06,4917.810014,6.080757,28.992411,209.900863,0.198273,1205.975590
std,179.112913,259.917002,798.584848,31415.378176,458.777865,119.817811,102.452284,1391.941038,0.531699,222.113201,...,1.715719e+06,8.947681e+06,2.089893e+04,1.233842e+07,6366.517959,14.943464,4.220938,109.370253,0.398732,908.509944
min,0.000000,0.000000,2.000000,8942.000000,0.000000,0.000000,0.000000,4.000000,1.030000,0.000000,...,0.000000e+00,0.000000e+00,2.178341e+03,2.178341e+03,0.796007,0.000000,18.309167,95.470254,0.000000,1.000000
25%,91.000000,281.000000,1013.000000,44504.500000,247.000000,101.000000,28.000000,1870.250000,2.350000,263.000000,...,6.139277e+04,1.732421e+06,5.894846e+03,3.022043e+06,1632.011003,0.000000,25.463787,163.361579,0.000000,575.000000
50%,167.000000,427.000000,1444.000000,58744.000000,424.000000,166.000000,65.000000,2665.000000,2.610000,394.000000,...,4.723928e+05,3.118269e+06,7.253804e+03,5.319199e+06,2718.112472,1.000000,30.932676,188.915562,0.000000,1176.500000
75%,273.000000,607.000000,1912.750000,81290.750000,715.000000,245.000000,145.000000,3577.000000,2.980000,556.750000,...,1.906141e+06,5.447937e+06,8.864285e+03,1.294382e+07,6034.326853,4.000000,30.932676,195.928370,0.000000,1717.750000
max,1660.000000,4982.000000,12316.000000,247868.000000,5777.000000,1307.000000,1163.000000,21953.000000,6.270000,2308.000000,...,1.461371e+07,1.657917e+08,1.144766e+06,1.720241e+08,104842.111072,258.000000,35.571604,477.108098,1.000000,12305.000000


In [10]:
data[data['weight']==0]

,12th_no_dip,Some_college,Total_households_by_type,Median_income,Bachelors,75k_100k,150k_200k,Pop_over_25,Avg_hh_size,Hh_w_children,...,yearly_sunlight_kwh_total,install_size_kw_buckets_json,carbon_offset_metric_tons,existing_installs_count,Region,Mean_price,Var_price,county_name,Project_yes,weight


In [11]:
acs_variables.append('Project_yes')
acs_variables.append('Mean_price')
acs_variables.append('Var_price')
acs_variables.append('percent_qualified')
acs_variables.append('yearly_sunlight_kwh_total')
acs_variables.append('number_of_panels_median')
acs_variables.append('kw_median')
acs_variables.append('weight')
X=data[acs_variables]
X.describe()


,Percent_Less_9th,Percent_12th_no_dip,Percent_HighSchool,Percent_Some_college,Percent_Associate,Percent_Bachelors,Percent_Graduate,Percent_more_than_hs,Avg_hh_size,Median_age,...,Percent_9_more_rooms,median_rooms,Project_yes,Mean_price,Var_price,percent_qualified,yearly_sunlight_kwh_total,number_of_panels_median,kw_median,weight
count,6022.000000,6022.000000,6022.000000,6022.000000,6022.000000,6022.000000,6022.000000,6022.000000,6016.000000,6015.000000,...,6022.000000,5968.000000,6022.000000,6022.000000,6022.000000,4334.000000,4.334000e+03,4334.000000,4334.000000,6022.000000
mean,0.061967,0.077948,0.279996,0.166751,0.089023,0.183827,0.140489,0.580089,2.691910,39.544472,...,0.101986,5.320409,0.198273,28.992411,209.900863,73.957591,9.921398e+06,35.464698,8.866174,1205.975590
std,0.063333,0.054221,0.100521,0.053400,0.041828,0.092332,0.105947,0.160649,0.531699,7.017340,...,0.091860,1.210531,0.398732,4.220938,109.370253,13.109499,1.233842e+07,72.975933,18.243983,908.509944
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.030000,11.300000,...,0.000000,1.300000,0.000000,18.309167,95.470254,21.212121,2.178341e+03,8.000000,2.000000,1.000000
25%,0.019415,0.037299,0.218787,0.133193,0.058894,0.114487,0.063992,0.462997,2.350000,34.700000,...,0.033550,4.200000,0.000000,25.463787,163.361579,66.166192,3.022043e+06,20.000000,5.000000,575.000000
50%,0.038274,0.067073,0.286477,0.167410,0.086880,0.169615,0.111104,0.572390,2.610000,40.300000,...,0.085869,5.500000,0.000000,30.932676,188.915562,75.756407,5.319199e+06,25.000000,6.250000,1176.500000
75%,0.082944,0.105995,0.350874,0.198676,0.116209,0.238077,0.187383,0.688009,2.980000,44.400000,...,0.144573,6.200000,0.000000,30.932676,195.928370,83.435121,1.294382e+07,31.000000,7.750000,1717.750000
max,0.445956,0.400000,0.793103,1.000000,1.000000,1.000000,1.000000,1.000000,6.270000,82.000000,...,1.000000,8.500000,1.000000,35.571604,477.108098,100.000000,1.720241e+08,3980.000000,995.000000,12305.000000


AttributeError: 'numpy.ndarray' object has no attribute 'weights'

In [ ]:
X.Project_yes.unique()

In [ ]:
np.mean(X.Project_yes)

In [12]:
y=X['Project_yes']
weights=X['weight']
X=X.drop(['Project_yes', 'weight'], axis=1)
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
X=imp.fit_transform(X)

In [155]:
y.unique()

AttributeError: 'DataFrame' object has no attribute 'unique'

In [47]:
X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(X, y, weights, test_size=0.3, random_state=42)

In [ ]:
model = GradientBoostingClassifier()
search = GridSearchCV(model, param_grid = {'n_estimators' : [500, 1000], 'subsample' :[0.5,0.8],
                                           'max_depth': [2,3,4,5], 'learning_rate' : [0.01, 0.001]}, 
             scoring='recall', fit_params={'sample_weight': weights_train}, n_jobs=1, iid=True, refit=True, 
             cv=None, verbose=5, pre_dispatch='2*n_jobs', error_score='raise', return_train_score=True)

In [ ]:
fitted = search.fit(X_train,y_train)

In [ ]:
y_train

In [ ]:
fitted.best_score_

In [ ]:
fitted.best_params_

In [60]:
model = GradientBoostingClassifier(learning_rate=0.01, max_depth=5, n_estimators=1000, subsample=0.8)

In [61]:
fitted = model.fit(X_train,y_train, sample_weight= weights_train)

In [62]:
fitted.score(X_test,y_test, sample_weight= weights_test)

0.97800922205597063

In [63]:
fitted.feature_importances_

array([ 0.01125359,  0.01193536,  0.00691949,  0.01450805,  0.03683914,
        0.00644373,  0.00617734,  0.00447493,  0.01172338,  0.00739624,
        0.00726872,  0.01575744,  0.0155557 ,  0.02488798,  0.01231154,
        0.00540407,  0.00738767,  0.0051124 ,  0.00762908,  0.01902951,
        0.04318098,  0.01435595,  0.01466155,  0.23945885,  0.04912324,
        0.00903821,  0.00317703,  0.00568943,  0.00486486,  0.00583531,
        0.01017777,  0.02238551,  0.01863945,  0.0120701 ,  0.01972542,
        0.11849364,  0.04150483,  0.02431001,  0.04362377,  0.01083976,
        0.00120708,  0.01216293,  0.02852198,  0.00514694,  0.0037901 ])

In [36]:
sum(np.unique(s)[:-3])

0.98938074855876335

In [55]:
n_features = len(fitted.feature_importances_)

In [86]:
foo = []
for i, tree in enumerate(fitted.estimators_):
    for tree[0] 
    if len(tree[0].feature_importances_) != n_features:
        foo.append(i)
        

SyntaxError: invalid syntax (<ipython-input-86-dd508eae933c>, line 3)

In [85]:
foo

[]

In [87]:
feature_importaces = np.array([
    tree[0].feature_importances_
    for tree in fitted.estimators_
])

In [88]:
feature_importaces

(1000, 45)

In [96]:
np.where(np.isnan(feature_importaces.mean(axis=1)))

(array([105, 634, 924]),)

In [92]:
from collections import Counter

In [109]:
badtree = fitted.estimators_[105][0].tree_

In [110]:
badtree.threshold

array([  5.84337950e-01,   1.21336006e-01,   3.37783366e-01,
         2.70738676e-02,   3.98966908e-01,  -2.00000000e+00,
        -2.00000000e+00,   1.72126830e-01,  -2.00000000e+00,
        -2.00000000e+00,   1.76958233e-01,  -2.00000000e+00,
         3.15964520e-01,  -2.00000000e+00,  -2.00000000e+00,
         6.26566412e-04,  -2.00000000e+00,  -2.00000000e+00,
        -2.00000000e+00])

In [111]:
badtree.children_left

array([ 1,  2,  3,  4,  5, -1, -1,  8, -1, -1, 11, -1, 13, -1, -1, 16, -1,
       -1, -1])

In [112]:
badtree.children_right

array([18, 15, 10,  7,  6, -1, -1,  9, -1, -1, 12, -1, 14, -1, -1, 17, -1,
       -1, -1])

In [120]:
badtree.feature

array([35, 24, 36, 25, 35, -2, -2, 17, -2, -2,  2, -2,  3, -2, -2, 18, -2,
       -2, -2])

In [118]:
badtree.impurity

array([  9.56628104e-03,   9.58454798e-03,   9.57655162e-03,
         9.52004760e-03,   7.86530018e-03,   7.85801884e-03,
         1.04416475e-13,   1.89902571e-02,   1.89714492e-02,
                    nan,   9.67752724e-02,   4.91632373e-02,
         6.26543914e-06,   1.54507961e-07,  -4.38017678e-17,
         1.86003734e-05,  -1.11022302e-16,  -1.88737914e-15,
        -5.49769888e+00])

In [128]:
badtree.weighted_n_node_samples

array([  4.06031200e+06,   4.06027300e+06,   4.06022000e+06,
         4.05796700e+06,   3.45711900e+06,   3.45696600e+06,
         1.53000000e+02,   6.00848000e+05,   6.00848000e+05,
         0.00000000e+00,   2.25300000e+03,   4.05000000e+02,
         1.84800000e+03,   1.84100000e+03,   7.00000000e+00,
         5.30000000e+01,   4.90000000e+01,   4.00000000e+00,
         3.90000000e+01])

In [78]:
np.argsort(feature_importaces[:,6])

array([  0, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663,
       651, 664, 666, 667, 668, 669, 670, 671, 672, 674, 677, 678, 679,
       680, 665, 681, 649, 647, 617, 618, 619, 620, 621, 623, 624, 625,
       626, 627, 628, 629, 648, 630, 632, 633, 635, 636, 638, 639, 640,
       642, 643, 644, 645, 646, 631, 682, 683, 684, 718, 719, 720, 721,
       723, 724, 725, 727, 728, 729, 730, 731, 717, 732, 734, 735, 736,
       737, 738, 739, 740, 741, 742, 743, 744, 745, 733, 716, 715, 714,
       685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 698,
       699, 700, 701, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712,
       713, 615, 746, 614, 612, 520, 521, 522, 523, 524, 525, 526, 527,
       528, 529, 530, 531, 518, 532, 534, 535, 537, 538, 539, 540, 541,
       542, 543, 544, 545, 546, 533, 547, 517, 515, 486, 487, 488, 489,
       490, 491, 492, 493, 494, 495, 496, 498, 516, 998, 501, 502, 503,
       504, 505, 507, 508, 509, 510, 511, 512, 514, 500, 548, 54

In [79]:
feature_importaces[:,6][634]

nan

In [84]:
fitted.estimators_[634][0].feature_importances_ == 0

array([ True,  True,  True,  True,  True, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,  True,  True,  True,  True, False,  True,  True], dtype=bool)

In [48]:
np.sum(fitted.estimators_[3][0].feature_importances_)

1.0000000000000002

In [57]:
len(foo)

0

In [41]:
fitted.estimators_[0][0]

DecisionTreeRegressor(criterion='friedman_mse', max_depth=5,
           max_features=None, max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort='auto',
           random_state=<mtrand.RandomState object at 0x10e8df0f0>,
           splitter='best')

In [38]:
len(fitted.estimators_)

1000

In [ ]:
from sklearn.metrics import recall_score

recall_score(y, fitted.predict(X))

In [ ]:
fitted.feature_importances_
feature_importances = 100*fitted.feature_importances_ / np.sum(fitted.feature_importances_)
feature_importances, feature_names, feature_idxs = zip(*sorted(zip(feature_importances, list(X.columns), range(len(X.columns)))))

width = 0.8

idx = np.arange(len(X.columns))

fig, ax = plt.subplots(figsize=(10,12))
ax.barh(idx, feature_importances, align='center')
plt.yticks(idx, feature_names, fontsize=8)

ax.set_title("Feature Importances in Gradient Booster")
ax.set_xlabel('Relative Importance of Feature', fontsize=10)
ax.set_ylabel('Feature Name', fontsize=14)
fig.savefig('feat_imp_sep8.png')

In [ ]:
list(X.columns)

In [ ]:
X2_variables = ['Percent_Less_9th','Percent_more_than_hs', 'Percent_1_unit','Avg_hh_size',
                'Percent_hh_w_children', 'Percent_10+_units', 'Percent_Heating_electricity',
                'yearly_sunlight_kwh_total'  , 'Percent_6_rooms', 'percent_qualified',
                'Project_yes','weight']

In [ ]:
len(X2_variables)

In [ ]:
X2=data[X2_variables]


In [ ]:
X2.head()

In [ ]:
y=X2['Project_yes']
weights=X2['weight']
X3=X2.drop(['Project_yes', 'weight'], axis=1)
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
X4 =imp.fit_transform(X3)

In [ ]:
X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(X4, y, weights, test_size=0.3, random_state=42)

In [ ]:
model = GradientBoostingClassifier(learning_rate=0.01, max_depth=5, n_estimators=1000, subsample=0.5)

In [ ]:
X_train.head()

In [ ]:
fitted = model.fit(X_train,y_train, sample_weight= weights_train)

In [ ]:
fitted.score(X_test,y_test, sample_weight= weights_test)

In [ ]:
len(fitted.feature_importances_)

In [ ]:
from sklearn.metrics import recall_score

recall_score(y_test, fitted.predict(X_test))

In [ ]:
fitted.feature_importances_

In [ ]:
fitted.feature_importances_
feature_importances = 100*fitted.feature_importances_ / np.sum(fitted.feature_importances_)
feature_importances, feature_names, feature_idxs = zip(*sorted(zip(feature_importances, list(X3.columns), range(len(X3.columns)))))

width = 0.8

idx = np.arange(len(X3.columns))

fig, ax = plt.subplots(figsize=(10,12))
ax.barh(idx, feature_importances, align='center')
plt.yticks(idx, feature_names, fontsize=8)

ax.set_title("Feature Importances in Gradient Booster")
ax.set_xlabel('Relative Importance of Feature', fontsize=10)
ax.set_ylabel('Feature Name', fontsize=14)
fig.savefig('feat_imp_sep11.png')

In [ ]:
model = GradientBoostingClassifier()
search = GridSearchCV(model, param_grid = {'n_estimators' : [500, 1000], 'subsample' :[0.5,0.8],
                                           'max_depth': [2,3,4,5], 'learning_rate' : [0.01, 0.001]}, 
             scoring='recall', fit_params={'sample_weight': weights_train}, n_jobs=1, iid=True, refit=True, 
             cv=None, verbose=5, pre_dispatch='2*n_jobs', error_score='raise', return_train_score=True)

In [ ]:
fitted = search.fit(X_train,y_train)

In [ ]:
fitted.score(X_train,y_train)

In [ ]:
fitted.best_params_

In [ ]:
result = fitted.predict_proba(X4)[:,1]

In [ ]:
X2['result']=result

In [ ]:
X2['GEOID10']=acs.GEOID10

In [ ]:
X2.to_csv('sept11result.csv')